# Tutorial 03: Running RLlib Experiments

This tutorial walks you through the process of running traffic simulations in Flow with trainable RLlib-powered agents. Autonomous agents will learn to maximize a certain reward over the rollouts, using the [**RLlib**](https://ray.readthedocs.io/en/latest/rllib.html) library ([citation](https://arxiv.org/abs/1712.09381)) ([installation instructions](https://flow.readthedocs.io/en/latest/flow_setup.html#optional-install-ray-rllib)). Simulations of this form will depict the propensity of RL agents to influence the traffic of a human fleet in order to make the whole fleet more efficient (for some given metrics). 

In this tutorial, we simulate an initially perturbed single lane ring road, where we introduce a single autonomous vehicle. We witness that, after some training, that the autonomous vehicle learns to dissipate the formation and propagation of "phantom jams" which form when only human driver dynamics are involved.

## 1. Components of a Simulation
All simulations, both in the presence and absence of RL, require two components: a *network*, and an *environment*. Networks describe the features of the transportation network used in simulation. This includes the positions and properties of nodes and edges constituting the lanes and junctions, as well as properties of the vehicles, traffic lights, inflows, etc... in the network. Environments, on the other hand, initialize, reset, and advance simulations, and act as the primary interface between the reinforcement learning algorithm and the network. Moreover, custom environments may be used to modify the dynamical features of an network. Finally, in the RL case, it is in the *environment* that the state/action spaces and the reward function are defined. 

## 2. Setting up a Network
Flow contains a plethora of pre-designed networks used to replicate highways, intersections, and merges in both closed and open settings. All these networks are located in flow/networks. For this tutorial, which involves a single lane ring road, we will use the network `RingNetwork`.

### 2.1 Setting up Network Parameters

The network mentioned at the start of this section, as well as all other networks in Flow, are parameterized by the following arguments: 
* name
* vehicles
* net_params
* initial_config

These parameters are explained in detail in `tutorial01_sumo.ipynb`. Moreover, all parameters excluding vehicles (covered in section 2.2) do not change from the previous tutorial. Accordingly, we specify them nearly as we have before, and leave further explanations of the parameters to `tutorial01_sumo.ipynb`.

We begin by choosing the network the experiment will be trained on. We use one of Flow's builtin networks, located in `flow.networks`. A list of all available networks can be found by running the script below.

In [1]:
import flow.networks as networks

print(networks.__all__)

['Network', 'BayBridgeNetwork', 'BayBridgeTollNetwork', 'BottleneckNetwork', 'FigureEightNetwork', 'TrafficLightGridNetwork', 'HighwayNetwork', 'RingNetwork', 'MergeNetwork', 'MultiRingNetwork', 'MiniCityNetwork', 'HighwayRampsNetwork', 'I210SubNetwork']


In this tutorial, we choose to use the ring road network. The network class is then:

In [2]:
from flow.networks import RingNetwork

# ring road network class
network_name = RingNetwork

One key difference between SUMO and RLlib experiments is that, in RLlib experiments, the network classes do not need to be defined; instead users should simply name the network class they wish to use. Later on, an environment setup module will import the correct network class based on the provided names.

In [3]:
# input parameter classes to the network class
from flow.core.params import NetParams, InitialConfig

# name of the network
name = "training_example"

# network-specific parameters
from flow.networks.ring import ADDITIONAL_NET_PARAMS
net_params = NetParams(additional_params=ADDITIONAL_NET_PARAMS)

# initial configuration to vehicles
initial_config = InitialConfig(spacing="uniform", perturbation=1)

### 2.2 Adding Trainable Autonomous Vehicles
The `Vehicles` class stores state information on all vehicles in the network. This class is used to identify the dynamical features of a vehicle and whether it is controlled by a reinforcement learning agent. Morover, information pertaining to the observations and reward function can be collected from various `get` methods within this class.

The dynamics of vehicles in the `Vehicles` class can either be depicted by sumo or by the dynamical methods located in flow/controllers. For human-driven vehicles, we use the IDM model for acceleration behavior, with exogenous gaussian acceleration noise with std 0.2 m/s2 to induce perturbations that produce stop-and-go behavior. In addition, we use the `ContinousRouter` routing controller so that the vehicles may maintain their routes closed networks.

As we have done in `tutorial01_sumo.ipynb`, human-driven vehicles are defined in the `VehicleParams` class as follows:

In [4]:
# vehicles class
from flow.core.params import VehicleParams

# vehicles dynamics models
from flow.controllers import IDMController, ContinuousRouter

vehicles = VehicleParams()
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=21)

The above addition to the `Vehicles` class only accounts for 21 of the 22 vehicles that are placed in the network. We now add an additional trainable autuonomous vehicle whose actions are dictated by an RL agent. This is done by specifying an `RLController` as the acceleraton controller to the vehicle. 

In [5]:
from flow.controllers import RLController

Note that this controller serves primarirly as a placeholder that marks the vehicle as a component of the RL agent, meaning that lane changing and routing actions can also be specified by the RL agent to this vehicle.

We finally add the vehicle as follows, while again using the `ContinuousRouter` to perpetually maintain the vehicle within the network.

In [6]:
vehicles.add(veh_id="rl",
             acceleration_controller=(RLController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=1)

## 3. Setting up an Environment

Several environments in Flow exist to train RL agents of different forms (e.g. autonomous vehicles, traffic lights) to perform a variety of different tasks. The use of an environment allows us to view the cumulative reward simulation rollouts receive, along with to specify the state/action spaces.

Sumo envrionments in Flow are parametrized by three components:
* `SumoParams`
* `EnvParams`
* `Network`

### 3.1 SumoParams
`SumoParams` specifies simulation-specific variables. These variables include the length of any simulation step and whether to render the GUI when running the experiment. For this example, we consider a simulation step length of 0.1s and deactivate the GUI. 

**Note** For training purposes, it is highly recommanded to deactivate the GUI in order to avoid global slow down. In such case, one just needs to specify the following: `render=False`

In [7]:
from flow.core.params import SumoParams

sim_params = SumoParams(sim_step=0.1, render=False)

### 3.2 EnvParams

`EnvParams` specifies environment and experiment-specific parameters that either affect the training process or the dynamics of various components within the network. For the environment `WaveAttenuationPOEnv`, these parameters are used to dictate bounds on the accelerations of the autonomous vehicles, as well as the range of ring lengths (and accordingly network densities) the agent is trained on.

Finally, it is important to specify here the *horizon* of the experiment, which is the duration of one episode (during which the RL-agent acquire data). 

In [8]:
from flow.core.params import EnvParams

# Define horizon as a variable to ensure consistent use across notebook
HORIZON=100

env_params = EnvParams(
    # length of one rollout
    horizon=HORIZON,

    additional_params={
        # maximum acceleration of autonomous vehicles
        "max_accel": 1,
        # maximum deceleration of autonomous vehicles
        "max_decel": 1,
        # bounds on the ranges of ring road lengths the autonomous vehicle 
        # is trained on
        "ring_length": [220, 270],
    },
)

### 3.3 Initializing a Gym Environment

Now, we have to specify our Gym Environment and the algorithm that our RL agents will use. Similar to the network, we choose to use on of Flow's builtin environments, a list of which is provided by the script below.

In [9]:
import flow.envs as flowenvs

print(flowenvs.__all__)

['Env', 'AccelEnv', 'LaneChangeAccelEnv', 'LaneChangeAccelPOEnv', 'TrafficLightGridTestEnv', 'MergePOEnv', 'BottleneckEnv', 'BottleneckAccelEnv', 'WaveAttenuationEnv', 'WaveAttenuationPOEnv', 'TrafficLightGridEnv', 'TrafficLightGridPOEnv', 'TrafficLightGridBenchmarkEnv', 'BottleneckDesiredVelocityEnv', 'TestEnv', 'BayBridgeEnv', 'BottleNeckAccelEnv', 'DesiredVelocityEnv', 'PO_TrafficLightGridEnv', 'GreenWaveTestEnv']


We will use the environment "WaveAttenuationPOEnv", which is used to train autonomous vehicles to attenuate the formation and propagation of waves in a partially observable variable density ring road. To create the Gym Environment, the only necessary parameters are the environment name plus the previously defined variables. These are defined as follows:

In [10]:
from flow.envs import WaveAttenuationPOEnv

env_name = WaveAttenuationPOEnv

### 3.4 Setting up Flow Parameters

RLlib experiments both generate a `params.json` file for each experiment run. For RLlib experiments, the parameters defining the Flow network and environment must be stored as well. As such, in this section we define the dictionary `flow_params`, which contains the variables required by the utility function `make_create_env`. `make_create_env` is a higher-order function which returns a function `create_env` that initializes a Gym environment corresponding to the Flow network specified.

In [11]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict(
    # name of the experiment
    exp_tag=name,
    # name of the flow environment the experiment is running on
    env_name=env_name,
    # name of the network class the experiment uses
    network=network_name,
    # simulator that is used by the experiment
    simulator='traci',
    # simulation-related parameters
    sim=sim_params,
    # environment related parameters (see flow.core.params.EnvParams)
    env=env_params,
    # network-related parameters (see flow.core.params.NetParams and
    # the network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,
    # vehicles to be placed in the network at the start of a rollout 
    # (see flow.core.vehicles.Vehicles)
    veh=vehicles,
    # (optional) parameters affecting the positioning of vehicles upon 
    # initialization/reset (see flow.core.params.InitialConfig)
    initial=initial_config
)

## 4 Running RL experiments in Ray

### 4.1 Import 

First, we must import modules required to run experiments in Ray. The `json` package is required to store the Flow experiment parameters in the `params.json` file, as is `FlowParamsEncoder`. Ray-related imports are required: the PPO algorithm agent, `ray.tune`'s experiment runner, and environment helper methods `register_env` and `make_create_env`.

In [12]:
import json

import ray
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

### 4.2 Initializing Ray
Here, we initialize Ray and experiment-based constant variables specifying parallelism in the experiment as well as experiment batch size in terms of number of rollouts.

In [13]:
# number of parallel workers
N_CPUS = 10
# number of rollouts per training iteration
N_ROLLOUTS = 1

ray.init(num_cpus=N_CPUS)

2022-09-04 12:11:11,374	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.13
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8265


### 4.3 Configuration and Setup
Here, we copy and modify the default configuration for the [PPO algorithm](https://arxiv.org/abs/1707.06347). The agent has the number of parallel workers specified, a batch size corresponding to `N_ROLLOUTS` rollouts (each of which has length `HORIZON` steps), a discount rate $\gamma$ of 0.999, two hidden layers of size 16, uses Generalized Advantage Estimation, $\lambda$ of 0.97, and other parameters as set below.

Once `config` contains the desired parameters, a JSON string corresponding to the `flow_params` specified in section 3 is generated. The `FlowParamsEncoder` maps objects to string representations so that the experiment can be reproduced later. That string representation is stored within the `env_config` section of the `config` dictionary. Later, `config` is written out to the file `params.json`. 

Next, we call `make_create_env` and pass in the `flow_params` to return a function we can use to register our Flow environment with Gym. 

In [14]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = PPOTrainer
config = DEFAULT_CONFIG.copy()
config["num_workers"] = N_CPUS - 1  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate
config["model"].update({"fcnet_hiddens": [16, 16]})  # size of hidden layers in network
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon
config["framework"] = "tf2"
config["eager_tracing"] = True

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

### 4.4 Running Experiments

Here, we use the `run_experiments` function from `ray.tune`. The function takes a dictionary with one key, a name corresponding to the experiment, and one value, itself a dictionary containing parameters for training.

In [15]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "checkpoint_at_end": True,  # generate a checkpoint at the end
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1,  # number of iterations to stop after
        },
    },
})

Trial name,status,loc
PPO_WaveAttenuationPOEnv-v0_31c09_00000,RUNNING,
Trial name,# failures,error file
PPO_WaveAttenuationPOEnv-v0_31c09_00000,45,/Users/maxdumas/ray_results/training_example/PPO_WaveAttenuationPOEnv-v0_31c09_00000_0_2022-09-04_12-11-12/error.txt


(PPO pid=62568) 2022-09-04 12:11:15,279	INFO algorithm.py:1871 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(PPO pid=62568) 2022-09-04 12:11:15,279	WARNING ppo.py:350 -- `train_batch_size` (100) cannot be achieved with your other settings (num_workers=9 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 12.
(PPO pid=62568) 2022-09-04 12:11:15,279	INFO ppo.py:378 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPO pid=62568) 2022-09-04 12:11:15,280	INFO algorithm.py:351 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=62573) 
(RolloutWorker pid=62573) -----------------------
(RolloutWorker pid=62573) ring length: 230
(RolloutWorker pid=62573) v_max: 3.7136148111012934
(RolloutWorker pid=62573) -----------------------
(RolloutWorker pid=62574) 
(RolloutWorker pid=62574) -----------------------
(RolloutWorker pid=62574) ring length: 247
(RolloutWorker pid=62574) v_max: 4.522125250095652
(RolloutWorker pid=62574) -----------------------
(RolloutWorker pid=62576) 
(RolloutWorker pid=62576) -----------------------
(RolloutWorker pid=62576) ring length: 221
(RolloutWorker pid=62576) v_max: 3.285334164169417
(RolloutWorker pid=62576) -----------------------
(RolloutWorker pid=62575) 
(RolloutWorker pid=62575) -----------------------
(RolloutWorker pid=62575) ring length: 250
(RolloutWorker pid=62575) v_max: 4.664717904914125
(RolloutWorker pid=62575) -----------------------
(RolloutWorker pid=62580) 
(RolloutWorker pid=62580) -----------------------
(RolloutWorker pid=62580) ring length:

(PPO pid=62568) 2022-09-04 12:11:21,370	ERROR worker.py:756 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=62568, ip=127.0.0.1, repr=PPO)
(PPO pid=62568)   File "/opt/homebrew/Caskroom/mambaforge/base/envs/flow-m1/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 125, in __init__
(PPO pid=62568)     self.add_workers(
(PPO pid=62568)   File "/opt/homebrew/Caskroom/mambaforge/base/envs/flow-m1/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 269, in add_workers
(PPO pid=62568)     self.foreach_worker(lambda w: w.assert_healthy())
(PPO pid=62568)   File "/opt/homebrew/Caskroom/mambaforge/base/envs/flow-m1/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 391, in foreach_worker
(PPO pid=62568)     remote_results = ray.get([w.apply.remote(func) for w in self.remote_workers()])
(PPO pid=62568) ray.exceptions.RayActorError: The actor died because of an e

(RolloutWorker pid=62626) 
(RolloutWorker pid=62626) -----------------------
(RolloutWorker pid=62626) ring length: 243
(RolloutWorker pid=62626) v_max: 4.331956438196479
(RolloutWorker pid=62626) -----------------------
(RolloutWorker pid=62628) 
(RolloutWorker pid=62628) -----------------------
(RolloutWorker pid=62628) ring length: 228
(RolloutWorker pid=62628) v_max: 3.618452967700356
(RolloutWorker pid=62628) -----------------------
(RolloutWorker pid=62629) 
(RolloutWorker pid=62629) -----------------------
(RolloutWorker pid=62629) ring length: 239
(RolloutWorker pid=62629) v_max: 4.141741239205832
(RolloutWorker pid=62629) -----------------------
(RolloutWorker pid=62625) 
(RolloutWorker pid=62625) -----------------------
(RolloutWorker pid=62625) ring length: 235
(RolloutWorker pid=62625) v_max: 3.9514850725282584
(RolloutWorker pid=62625) -----------------------
(RolloutWorker pid=62624) 
(RolloutWorker pid=62624) -----------------------
(RolloutWorker pid=62624) ring length:

...

(PPO pid=64732) 2022-09-04 12:18:41,304	ERROR worker.py:756 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=64732, ip=127.0.0.1, repr=PPO)
(PPO pid=64732)   File "/opt/homebrew/Caskroom/mambaforge/base/envs/flow-m1/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 125, in __init__
(PPO pid=64732)     self.add_workers(
(PPO pid=64732)   File "/opt/homebrew/Caskroom/mambaforge/base/envs/flow-m1/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 269, in add_workers
(PPO pid=64732)     self.foreach_worker(lambda w: w.assert_healthy())
(PPO pid=64732)   File "/opt/homebrew/Caskroom/mambaforge/base/envs/flow-m1/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 391, in foreach_worker
(PPO pid=64732)     remote_results = ray.get([w.apply.remote(func) for w in self.remote_workers()])
(PPO pid=64732) ray.exceptions.RayActorError: The actor died because of an e

(RolloutWorker pid=64781) 
(RolloutWorker pid=64781) -----------------------
(RolloutWorker pid=64781) ring length: 235
(RolloutWorker pid=64781) v_max: 3.9514850725282584
(RolloutWorker pid=64781) -----------------------
(RolloutWorker pid=64787) 
(RolloutWorker pid=64787) -----------------------
(RolloutWorker pid=64787) ring length: 237
(RolloutWorker pid=64787) v_max: 4.04661795519353
(RolloutWorker pid=64787) -----------------------
(RolloutWorker pid=64780) 
(RolloutWorker pid=64780) -----------------------
(RolloutWorker pid=64780) ring length: 226
(RolloutWorker pid=64780) v_max: 3.5232840694769565
(RolloutWorker pid=64780) -----------------------
(RolloutWorker pid=64783) 
(RolloutWorker pid=64783) -----------------------
(RolloutWorker pid=64783) ring length: 256
(RolloutWorker pid=64783) v_max: 4.949804327743507
(RolloutWorker pid=64783) -----------------------
(RolloutWorker pid=64786) 
(RolloutWorker pid=64786) -----------------------
(RolloutWorker pid=64786) ring length:

(PPO pid=64779) 2022-09-04 12:18:51,471	ERROR worker.py:756 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=64779, ip=127.0.0.1, repr=PPO)
(PPO pid=64779)   File "/opt/homebrew/Caskroom/mambaforge/base/envs/flow-m1/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 125, in __init__
(PPO pid=64779)     self.add_workers(
(PPO pid=64779)   File "/opt/homebrew/Caskroom/mambaforge/base/envs/flow-m1/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 269, in add_workers
(PPO pid=64779)     self.foreach_worker(lambda w: w.assert_healthy())
(PPO pid=64779)   File "/opt/homebrew/Caskroom/mambaforge/base/envs/flow-m1/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 391, in foreach_worker
(PPO pid=64779)     remote_results = ray.get([w.apply.remote(func) for w in self.remote_workers()])
(PPO pid=64779) ray.exceptions.RayActorError: The actor died because of an e

(RolloutWorker pid=64830) 
(RolloutWorker pid=64830) -----------------------
(RolloutWorker pid=64830) ring length: 234
(RolloutWorker pid=64830) v_max: 3.903915223349057
(RolloutWorker pid=64830) -----------------------
(RolloutWorker pid=64829) 
(RolloutWorker pid=64829) -----------------------
(RolloutWorker pid=64829) ring length: 244
(RolloutWorker pid=64829) v_max: 4.379503211387676
(RolloutWorker pid=64829) -----------------------
(RolloutWorker pid=64827) 
(RolloutWorker pid=64827) -----------------------
(RolloutWorker pid=64827) ring length: 270
(RolloutWorker pid=64827) v_max: 5.6143732387852054
(RolloutWorker pid=64827) -----------------------
(RolloutWorker pid=64832) 
(RolloutWorker pid=64832) -----------------------
(RolloutWorker pid=64832) ring length: 252
(RolloutWorker pid=64832) v_max: 4.759762049824181
(RolloutWorker pid=64832) -----------------------
(RolloutWorker pid=64828) 
(RolloutWorker pid=64828) -----------------------
(RolloutWorker pid=64828) ring length:

(PPO pid=64826) 2022-09-04 12:19:01,487	ERROR worker.py:756 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=64826, ip=127.0.0.1, repr=PPO)
(PPO pid=64826)   File "/opt/homebrew/Caskroom/mambaforge/base/envs/flow-m1/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 125, in __init__
(PPO pid=64826)     self.add_workers(
(PPO pid=64826)   File "/opt/homebrew/Caskroom/mambaforge/base/envs/flow-m1/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 269, in add_workers
(PPO pid=64826)     self.foreach_worker(lambda w: w.assert_healthy())
(PPO pid=64826)   File "/opt/homebrew/Caskroom/mambaforge/base/envs/flow-m1/lib/python3.9/site-packages/ray/rllib/evaluation/worker_set.py", line 391, in foreach_worker
(PPO pid=64826)     remote_results = ray.get([w.apply.remote(func) for w in self.remote_workers()])
(PPO pid=64826) ray.exceptions.RayActorError: The actor died because of an e

### 4.5 Visualizing the results

The simulation results are saved within the `ray_results/training_example` directory (we defined `training_example` at the start of this tutorial). The `ray_results` folder is by default located at your root `~/ray_results`. 

You can run `tensorboard --logdir=~/ray_results/training_example` (install it with `pip install tensorboard`) to visualize the different data outputted by your simulation.

For more instructions about visualizing, please see `tutorial05_visualize.ipynb`. 

### 4.6 Restart from a checkpoint / Transfer learning

If you wish to do transfer learning, or to resume a previous training, you will need to start the simulation from a previous checkpoint. To do that, you can add a `restore` parameter in the `run_experiments` argument, as follows:

```python
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "restore": "/ray_results/experiment/dir/checkpoint_50/checkpoint-50"
        "checkpoint_freq": 1,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 1,
        },
    },
})
```

The `"restore"` path should be such that the `[restore]/.tune_metadata` file exists.

There is also a `"resume"` parameter that you can set to `True` if you just wish to continue the training from a previously saved checkpoint, in case you are still training on the same experiment. 